In [1]:
%pip install --upgrade --quiet langchain langchain-community langchainhub langchain-openai langchain-chroma bs4


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
!pip install bitsandbytes==0.40.0 einops==0.6.1
!pip install accelerate

The history saving thread hit an unexpected error (OperationalError('unable to open database file')).History will not be written to the database.

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
ERROR: Exception:
Traceback (most recent call last):
  File "/Users/wonik/.pyenv/versions/3.12.5/lib/python3.12/site-packages/pip/_internal/cli/base_command.py", line 105, in _run_wrapper
    status = _inner_run()
             ^^^^^^^^^^^^
  File "/Users/wonik/.pyenv/versions/3.12.5/lib/python3.12/site-packages/pip/_internal/cli/base_command.py", line 96, in _inner_run
    return self.run(options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/wonik/.pyenv/versions/3.12.5/lib/python3.12/site-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/wonik/.pyenv/versions/3.12.5/lib/python3.12/site-packages/pip/_internal/commands/ins

In [12]:
!pip install transformers


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
ERROR: Exception:
Traceback (most recent call last):
  File "/Users/wonik/.pyenv/versions/3.12.5/lib/python3.12/site-packages/pip/_internal/cli/base_command.py", line 105, in _run_wrapper
    status = _inner_run()
             ^^^^^^^^^^^^
  File "/Users/wonik/.pyenv/versions/3.12.5/lib/python3.12/site-packages/pip/_internal/cli/base_command.py", line 96, in _inner_run
    return self.run(options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/wonik/.pyenv/versions/3.12.5/lib/python3.12/site-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/wonik/.pyenv/versions/3.12.5/lib/python3.12/site-packages/pip/_internal/commands/install.py", line 334, in run
    build_tracker = self.enter_context(get_build_tracker())
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

1. indexing: load
DocumentLoaders를 사용하여 블로그 내용을 롣드.
Doucment는 page_content(str)와 메타데이터(dict)를 포함하는 객체
이 경우 urllib을 사용하여 웹 URL에서 HTML을 로드하고, BeautifulSoup을 사용하여 텍스트로 구문 분석
bs_kwags를 통해 BeautifulSoup 파서에 매개변수를 전달하여 HTML -> 텍스트 구문 분석을 사용자 정의할 ㅅ ㅜ있다.


In [14]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
import getpass 
import os
from dotenv import load_dotenv
from torch import cuda, bfloat16
import transformers

load_dotenv()
llm = ChatOpenAI(model="gpt-3.5-turbo")

model_id = 'meta-llama/Meta-Llama-3B'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# 양자화 설정으로 모델의 GPU 메모리 사용을 줄이도록 설정
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

hf_auth = os.environ["HUGGING_FACE_KEY"]
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
    use_auth_token=hf_auth
)

# post의 타이틀, 헤더, 콘텐츠만 가져오기
bs4_strainer = bs4.SoupStrainer(class_=("post-title", "post-header", "post-content"))
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs={"parse_only": bs4_strainer}
)
docs = loader.load()
print(docs[0].page_content[:500])

ImportError: tokenizers>=0.21,<0.22 is required for a normal functioning of this module, but found tokenizers==0.20.3.
Try: `pip install transformers -U` or `pip install -e '.[dev]'` if you're working with git main